In [1]:
from numpy.random import seed
seed(1)
import scipy.spatial as sp
from tensorflow import set_random_seed
set_random_seed(2)
import numpy as np, pandas as pd
from collections import Counter
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Add, concatenate , Subtract
from keras.callbacks import EarlyStopping
from keras.layers import Input, Dense, Dropout, BatchNormalization,Lambda
from keras.models import Model
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score,precision_score,recall_score
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.dummy import DummyClassifier
import json
# np.random.seed(44)
from sklearn.utils import shuffle
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))


/usr/lib64/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/lib64/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
Using TensorFlow backend.


In [3]:

def load_embeddings(path, dimension):
    f = open(path, encoding="utf8").read().splitlines()
    vectors = {}
    for i in f:
        elems = i.split()
        vectors[" ".join(elems[:-dimension])] =  np.array(elems[-dimension:]).astype(float)
    return vectors

In [4]:
embeddings = load_embeddings("glove.6B.300d.txt", 300)

In [5]:
# load embeddings and mark the words that are in the embeddings dictionary

def get_vector_representation_of_word_pairs(dataframe, embeddings_voca,concat,diff,Cosine):
    x1 = [embeddings_voca[word] for word in dataframe.w1.values]
    x2 =[embeddings_voca[word] for word in dataframe.w2.values]
    y = dataframe.Category.values
    #Concatenation
    if concat :
        x = np.hstack((x1, x2))
    if diff :
        z=np.subtract(x1, x2)
        if concat :
            x= np.hstack((x,z))
        else :
            x=z
    if Cosine :
        cosine=np.diag(1 - sp.distance.cdist(x1,x2, 'cosine'))
        cosine=np.reshape(cosine,(-1,1))
        x= np.hstack((x,cosine))
    

    return x, y



In [ ]:
#Load the data,  the Output is 3 Dataframes : df1 for task1 ,df2 for task2 and df3 for Random Pairs

def three_data(Rumen,Root9,Bless,Cogalex,Weeds):
    #Perpare_data for BLess or Root9
    def prep_df(df):
        cols =df.columns
        for i in range(len(cols)) :
            for j in range(len(df)):
                df[df.columns[i]].values[j]=df[df.columns[i]].values[j][0:-2]
        return df
    if Weeds:
        Rumen=True
        task1="HYPER"
        task2="Coord"
        link2="data/coordpairs2_wiki100.json"
        link1="data/entpairs2_wiki100.json"
    if Cogalex :
        task1="HYPER"
        task2="SYN"
        link1="CogALexV_train_v1/gold_task2.txt"
        link2="CogALexV_test_v1/gold_task2.txt"
    if Rumen :
        task1="HYPER"
        task2="SYN"
        link="RumenPairs.txt"
    if Root9 :
        link_hyper="MULTITASK-LEARNING/ROOT9/ROOT9_hyper.txt"
        link_coord="MULTITASK-LEARNING/ROOT9/ROOT9_coord.txt"
        link_random="MULTITASK-LEARNING/ROOT9/ROOT9_random.txt"
        task1= "HYPER"
        task2= "COORD"
    if Bless :
        task1= "HYPER"
        task2= "MERO"
        link_coord="MULTITASK-LEARNING/BLESS/BLESS_mero.txt"
        link_hyper="MULTITASK-LEARNING/BLESS/BLESS_hyper.txt"
        link_random="MULTITASK-LEARNING/BLESS/BLESS_random.txt"


    def get_names(cat):
        if cat == 0 : return "RANDOM"
        if cat == 1: return task1
        if cat == 2: return task2
    def get_names_Weeds1(cat):
        if cat == 0 : return "RANDOM"
        if cat == 1: return task1
    def get_names_Weeds2(cat):
        if cat == 0 : return "RANDOM"
        if cat == 1: return task2


    if Rumen :
        dff = pd.read_csv(link)
        dff.rename(columns={"W1":"w1", "W2":"w2","rel":"Category"}, inplace=True)
        dff["Category"] = dff["Category"].apply(get_names)
        df = dff.loc[dff.Category == task2]
        df2 = dff.loc[dff.Category == task1]
        df3 = dff.loc[dff.Category == "RANDOM"]
        #print(len(df),len(df2),len(df3))
    if Root9 or Bless:

        df = pd.read_csv(link_coord,header=None,sep = '\t')
        df.rename(index=str,columns={0:"w1", 2:"w2",1:"Category"},inplace=True)
        df=prep_df(df)
        df2 = pd.read_csv(link_hyper,header=None,sep = '\t')
        df2.rename(index=str,columns={0:"w1", 2:"w2",1:"Category"},inplace=True)
        df2=prep_df(df2)
        df3 = pd.read_csv(link_random,header=None,sep = '\t')
        df3.rename(index=str,columns={0:"w1", 2:"w2",1:"Category"},inplace=True)
        df3=prep_df(df3)
    if Cogalex:

        dff1 = pd.read_csv(link1,header=None,sep = '\t')
        dff2 = pd.read_csv(link2,header=None,sep = '\t')
        dff1.rename(index=str,columns={0:"w1", 1:"w2",2:"Category"},inplace=True)
        dff2.rename(index=str,columns={0:"w1", 1:"w2",2:"Category"},inplace=True)
        dff3=pd.concat([dff1,dff2])
        #print(list(set(dff3.Category.values.tolist())))
        df = dff3.loc[dff3.Category == task2]
        df2 = dff3.loc[dff3.Category == task1]
        df3 = dff3.loc[dff3.Category == "RANDOM"]
    if Weeds :
        json_data=open(link1).read()
        data = json.loads(json_data)
        dff=pd.DataFrame(data)
        dff.rename(index=str,columns={0:"w1", 1:"w2",2:"Category"},inplace=True)
        dff["Category"] = dff["Category"].apply(get_names_Weeds1)
        df2 = dff.loc[dff.Category == task1]
        #df3 = dff.loc[dff.Category == "RANDOM"]
        #df3=df3[0:len(df2)]
        #print("taille 0,1 pour entpairs",len(df2),len(df3))
        
        json_data2=open(link2).read()
        data2 = json.loads(json_data2)
        dff2=pd.DataFrame(data2)
        dff2.rename(index=str,columns={0:"w1", 1:"w2",2:"Category"},inplace=True)
        dff2["Category"] = dff2["Category"].apply(get_names_Weeds2)
        df = dff2.loc[dff2.Category == task2]
        #df=df[0:len(df2)]
        #print("taille 0,1 pour coord",len(df))
        
        
    return df,df2,df3



In [ ]:
def func_train_test(name_data,embeddings,concat,diff,cosine):
    
    Cogalex,Rumen,Root9,Weeds,Bless=False,False,False,False,False
    if str(name_data)=="Cogalex" :
        Cogalex=True
        #print('Preprocessing of Cogalex ')
    if str(name_data)=="Weeds" :
        Weeds=True
        #print('Preprocessing of Weeds ')
        
    if str(name_data)=="Rumen" :
        Rumen=True
        #print('Preprocessing of Rumen ')
    if str(name_data)=="Root9" :
        Root9=True
        #print('Preprocessing of Root9 ')
    if str(name_data)=="Bless" :
        Bless=True
        #print('Preprocessing of Bless')
        
    df,df2,df3=three_data(Rumen,Root9,Bless,Cogalex,Weeds)
 
    words_coord = list(set(df.w1.values.tolist() + df.w2.values.tolist()))
    words_hyper = list(set(df2.w1.values.tolist() + df2.w2.values.tolist()))
    words_random = list(set(df3.w1.values.tolist() + df3.w2.values.tolist()))

    words_ = sorted(list(set(words_coord+words_hyper+words_random)))
    #words_ = sorted(list(set(words_coord+words_hyper+words_random1+words_random2)))
    words_train, words_test =train_test_split(words_, test_size=0.4, random_state=1344)
    #print('numbers_words_task1_not_sorted: '+str(len(list((df.w1.values.tolist() + df.w2.values.tolist())))))
    #print('numbers_words_sorted: '+str(len(words_)) )
    #print('numbers_words_task1_sorted: '+str(len(words_coord)))
    #print('numbers_words_task2_sorted: '+str(len(words_hyper)) )
    #print('numbers_words_random_sorted: '+str(len(words_random)) +'\n')

    
    #print('numbers_pairs_task1_all: '+str(len(df)) )
    #print('numbers_pairs_task2_all: '+str(len(df2)) )
    #print('numbers_pairs_random_all: '+str(len(df3)) +'\n')
  

    df["known_words"] = df.apply(lambda l: l["w1"] in embeddings and l["w2"] in embeddings, axis =1  )
    df2["known_words"] = df2.apply(lambda l: l["w1"] in embeddings and l["w2"] in embeddings, axis =1  )
    df3["known_words"] = df3.apply(lambda l: l["w1"] in embeddings and l["w2"] in embeddings, axis =1  )
    
    
    #print('numbers_pairs_task1_known: '+str(len(df)) )
    #print('numbers_pairs_task2_known: '+str(len(df2)) )
    #print('numbers_pairs_random_known: '+str(len(df3)) +'\n')
    
    
    # Given the words in the train and test parts, mark the pairs as training or testing, when both words of aa pair belong to the train or test vocabulary.
    df["is_train"] = df.apply(lambda l : l["w1"] in words_train and l["w2"] in words_train and l["known_words"] == True, axis=1 )
    df["is_test"] = df.apply(lambda l : l["w1"] in words_test and l["w2"] in words_test and l["known_words"] == True, axis=1)
 


    

    df2["is_train"] = df2.apply(lambda l : l["w1"] in words_train and l["w2"] in words_train and l["known_words"] == True, axis=1 )
    df2["is_test"] = df2.apply(lambda l : l["w1"] in words_test and l["w2"] in words_test and l["known_words"] == True, axis=1)
 
    
    df3["is_train"] = df3.apply(lambda l : l["w1"] in words_train and l["w2"] in words_train and l["known_words"] == True, axis=1 )
    df3["is_test"] = df3.apply(lambda l : l["w1"] in words_test and l["w2"] in words_test and l["known_words"] == True, axis=1)

    xtrainCoord, ytrainCoord = get_vector_representation_of_word_pairs(df.loc[df.is_train==True], embeddings,concat,diff,cosine)
    xtestCoord, ytestCoord   = get_vector_representation_of_word_pairs(df.loc[df.is_test==True], embeddings,concat,diff,cosine)
    
    #print('numbers_pairs_task1_train: '+str(len(ytrainCoord) ))
    #print('numbers_pairs_task1_test: '+str(len(ytestCoord)) )
    #print('Percentage_pairs_train/test: '+str(len(ytestCoord)*100/(len(ytestCoord)+len(ytrainCoord)))+ '%'+'\n') 
    
    

    xtrainHyper, ytrainHyper = get_vector_representation_of_word_pairs(df2.loc[df2.is_train==True], embeddings,concat,diff,cosine)
    xtestHyper, ytestHyper   = get_vector_representation_of_word_pairs(df2.loc[df2.is_test==True], embeddings,concat,diff,cosine)
          
    #print('numbers_pairs_task2_train: '+str(len(ytrainHyper) ))
    #print('numbers_pairs_task2_test: '+str(len(ytestHyper)) )
    #print('Percentage_pairs_train/test: '+str(len(ytestHyper)*100/(len(ytestHyper)+len(ytrainHyper)))+ '%'+'\n') 
          
          
          
    xtrainRando, ytrainRando = get_vector_representation_of_word_pairs(df3.loc[df3.is_train==True], embeddings,concat,diff,cosine) 
    xtestRando, ytestRando   = get_vector_representation_of_word_pairs(df3.loc[df3.is_test==True], embeddings,concat,diff,cosine)
    
    #print('numbers_pairs_random_train: '+str(len(ytrainRando) ))
    #print('numbers_pairs_random_test: '+str(len(ytestRando)) )
    #print('Percentage_random_train/test: '+str(len(ytestRando)*100/(len(ytestRando)+len(ytrainRando)))+ '%'+'\n') 
          
    x_train_1, x_train_2 = np.vstack((xtrainCoord, xtrainRando)), np.vstack((xtrainHyper, xtrainRando))
    y_train_1, y_train_2 = [1]*len(xtrainCoord) + [0]*len(xtrainRando), [1]*len(xtrainHyper) + [0]*len(xtrainRando)



    x_test_1, x_test_2 = np.vstack((xtestCoord, xtestRando)), np.vstack((xtestHyper, xtestRando))
    y_test_1, y_test_2 = [1]*len(xtestCoord) + [0]*len(xtestRando), [1]*len(xtestHyper) + [0]*len(xtestRando)
    


    x_train_1, y_train_1 = shuffle(x_train_1, y_train_1, random_state=1234)
    x_train_2, y_train_2 = shuffle(x_train_2, y_train_2, random_state=1234)
    x_test_1, y_test_1 = shuffle(x_test_1, y_test_1, random_state=1234)
    x_test_2, y_test_2 = shuffle(x_test_2, y_test_2, random_state=1234)
    assert len(x_train_1) == len(y_train_1)
    assert len(x_train_2) == len(y_train_2)
    assert len(x_test_1) == len(y_test_1)
    assert len(x_test_2) == len(y_test_2)
    data = {}
    for name, x_train, y_train, x_test, y_test in zip(["Coord-Random","Hyper-Random"], [x_train_1, x_train_2], [y_train_1, y_train_2], [x_test_1, x_test_2], [y_test_1, y_test_2]):   
        # Perform the splits in train, validation, unlabeled
        #x_train, x_unlabeled, y_train, y_unlabeled = train_test_split(x_train, y_train, stratify=y_train, test_size=0.6, random_state=1234,)
        x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, stratify=y_train,  test_size=0.30, random_state=1234,)


        # keep the train/validation/test splits so that hey can be used with multitask learning and the results are comparable between them
        #print('name '+name)
        data[name]={"x_train": x_train, "y_train":y_train,   "x_valid":x_valid, "y_valid":y_valid, "x_test":x_test,  "y_test":y_test } 
    
    sum_task1Vsrandom=len(data["Coord-Random"]["y_train"])+len(data["Coord-Random"]["y_test"])+len(data["Coord-Random"]["y_valid"])
    sum_task2Vsrandom=len(data["Hyper-Random"]["y_train"])+len(data["Hyper-Random"]["y_test"])+len(data["Hyper-Random"]["y_valid"])
    #print('sums_pairs_task1Vsrandom :' +str(sum_task1Vsrandom) )
    #print('sums_pairs_task2Vsrandom :' +str(sum_task2Vsrandom) +'\n')
        
    #print('numbers_pairs_task1Vsrandom_train: '+str(len(data["Coord-Random"]["y_train"]) ), 'Percentage :'+ str(len(data["Coord-Random"]["y_train"])*100/sum_task1Vsrandom) +' %')
    #print('numbers_pairs_task2Vsrandom_train: '+str(len(data["Hyper-Random"]["y_train"]) ), 'Percentage :'+ str(len(data["Hyper-Random"]["y_train"])*100/sum_task2Vsrandom) +' %'+'\n')
    
    #print('numbers_pairs_task1Vsrandom_valid: '+str(len(data["Coord-Random"]["y_valid"]) ),'Percentage :'+ str(len(data["Coord-Random"]["y_valid"])*100/sum_task1Vsrandom) +' %')
    #print('numbers_pairs_task2Vsrandom_valid: '+str(len(data["Hyper-Random"]["y_valid"]) ) , 'Percentage :'+ str(len(data["Hyper-Random"]["y_valid"])*100/sum_task2Vsrandom) +' %'+'\n')
    
    #print('numbers_pairs_task1Vsrandom_test: '+str(len(data["Coord-Random"]["y_test"]) ),'Percentage :'+ str(len(data["Coord-Random"]["y_test"])*100/sum_task1Vsrandom) +' %')
    #print('numbers_pairs_task2Vsrandom_test: '+str(len(data["Hyper-Random"]["y_test"]) ), 'Percentage :'+ str(len(data["Hyper-Random"]["y_test"])*100/sum_task2Vsrandom) +' %'+'\n')
    return data




In [8]:
def input_dim_func(concat,diff,cosine):
    if concat and diff== False  and cosine==False   :
        input_dim=600
    if concat and diff== False  and cosine   :
        input_dim=601
    if concat and diff and cosine==False   :
        input_dim=900
    if concat and diff and cosine   :
        input_dim=901
    if concat==False and diff and cosine==False   :
        input_dim=300
    if concat==False and diff  and cosine  :
        input_dim=301
    return input_dim

In [14]:

def get_my_nn_model():
    input_dim=input_dim_func(concat,diff,cosine)
    #print('NN_baseline','input_dim ',input_dim)
    """Defines the NN baseline.
    Two hidden layers, followed by the output layer. 
    """
    model = Sequential()
    model.add(Dense(50, activation='sigmoid', input_dim=input_dim))
    #model.add(Dropout(0.1))
    #model.add(Dense(15, activation='sigmoid'))

    model.add(Dense(1, activation='sigmoid'))
    #model.add(Dropout(0.1))
    model.compile(optimizer='Adam', loss='binary_crossentropy')
    return model

def get_my_multitask_nn_models_ancien():
    #print('Multi Ancien')
    input_dim=input_dim_func(concat,diff,cosine)
    inputs = Input(shape=(input_dim,))
    x = Dense(50, activation='sigmoid')(inputs)
    
   
    coord = Dense(1, activation='sigmoid', name='coord_output')(x)
    #coord=Dropout(0.05)(coord)
    hyper = Dense(1, activation='sigmoid', name='hyper_output')(x)
    #hyper=Dropout(0.5)(hyper)
    
    model_coord = Model(inputs=[inputs], outputs=[coord])
    model_hyper = Model(inputs=[inputs], outputs=[hyper])

    model_coord.compile(optimizer='rmsprop', loss='binary_crossentropy')
    model_hyper.compile(optimizer='rmsprop', loss='binary_crossentropy')
    return model_coord, model_hyper

def get_my_multitask_nn_models():
    #print('Multi New','input_dim ',input_dim)
    input_dim=input_dim_func(concat,diff,cosine)
    inputs = Input(shape=(input_dim,))
    #xcoord0 = Dense(50, activation='sigmoid',kernel_initializer='zeros',bias_initializer='zeros')(inputs)
    #xcoord = Dense(300, activation='sigmoid')(inputs)
    #X1=Dense(25, activation='sigmoid')(xcoord)
    x = Dense(300, activation='relu')(inputs)
    #xhyper0 = Dense(50, activation='sigmoid',kernel_initializer='zeros',bias_initializer='zeros')(inputs)
    #xhyper = Dense(300, activation='sigmoid')(inputs)
    #X2=Dense(25, activation='sigmoid')(xhyper)
    #model_co=Model(inputs,xcoord)
    #model_hy=Model(inputs,xhyper)
    
                

    #coord=Dropout(0.05)(coord)
 
          
    #shared=Dense(300, activation='sigmoid')   
    
    #shared=Dense(50, activation='sigmoid')(x)       
    #out_coord = shared(xcoord)
    #out_hyper = shared(xhyper)
    
    concatenated_hyper = concatenate([inputs,x])
    concatenated_coord = concatenate([inputs,x])
    #concatenated = Add()([xcoord,xhyper])
    #concatenated = Subtract()([xcoord,xhyper])
    xcoord = Dense(50, activation='sigmoid')(concatenated_coord)
    xhyper= Dense(50, activation='sigmoid')(concatenated_hyper)
    #xcommun= Dense(50,activation='sigmoid')(concatenated)
    #xafter= Dense(25,activation='sigmoid')(xcommun)
    
    
    #hyper0 = Dense(5, activation='sigmoid')(concatenated_hyper)
    hyper = Dense(1, activation='sigmoid', name='hyper_output')(xhyper)
    #hyper=Dropout(0.5)(hyper)

    
    model_hyper = Model(inputs=[inputs], outputs=[hyper])    
                

    #coord0 = Dense(5, activation='sigmoid')(concatenated_coord)
    coord = Dense(1, activation='sigmoid', name='coord_output')(xcoord)
    model_coord = Model(inputs=[inputs], outputs=[coord])
    
    



    model_coord.compile(optimizer='rmsprop', loss='binary_crossentropy')
    model_hyper.compile(optimizer='rmsprop', loss='binary_crossentropy')
    return model_coord, model_hyper


def get_my_multitask_nn_models_new():
    #print('Multi New2')
    input_dim=input_dim_func(concat,diff,cosine)
    inputs = Input(shape=(input_dim,))
    if input_dim<600 :
        model_coord, model_hyper= get_my_multitask_nn_models()
        return model_coord, model_hyper
    else :    
    #xcoord0 = Dense(50, activation='sigmoid',kernel_initializer='zeros',bias_initializer='zeros')(inputs)
    #xcoord = Dense(300, activation='sigmoid')(inputs)
    #X1=Dense(25, activation='sigmoid')(xcoord)
    #inputs1=inputs[0:300]
    #inputs2=inputs[300:600]
        inputs1=Lambda(lambda x: x[:,0:300])(inputs)

        inputs2=Lambda(lambda x: x[:,300:600])(inputs)
        inputs3=Lambda(lambda x: x[:,600:])(inputs)
        x = Dense(300, activation='relu')(inputs)
        x1= Dense(100, activation='relu')(inputs1)
        x2= Dense(100, activation='relu')(inputs2)
        #xhyper0 = Dense(50, activation='sigmoid',kernel_initializer='zeros',bias_initializer='zeros')(inputs)
        #xhyper = Dense(300, activation='sigmoid')(inputs)
        #X2=Dense(25, activation='sigmoid')(xhyper)
        #model_co=Model(inputs,xcoord)
        #model_hy=Model(inputs,xhyper)



        #coord=Dropout(0.05)(coord)


        #shared=Dense(300, activation='sigmoid')   

        #shared=Dense(50, activation='sigmoid')(x)       
        #out_coord = shared(xcoord)
        #out_hyper = shared(xhyper)

        concatenated_hyper = concatenate([inputs,x])       
        concatenated_hyper1 = concatenate([concatenated_hyper,x1])
        concatenated_hyper2 = concatenate([concatenated_hyper1,x2])
        concatenated_hyper3 = concatenate([concatenated_hyper2,inputs3])


        concatenated_coord = concatenate([inputs,x])
        concatenated_coord1 = concatenate([concatenated_coord,x1])
        concatenated_coord2 = concatenate([concatenated_coord1,x2])
        concatenated_coord3 = concatenate([concatenated_coord2,inputs3])
        #concatenated = Add()([xcoord,xhyper])
        #concatenated = Subtract()([xcoord,xhyper])
        xcoord = Dense(50, activation='sigmoid')(concatenated_coord3)
        xhyper= Dense(50, activation='sigmoid')(concatenated_hyper3)
        #xcommun= Dense(50,activation='sigmoid')(concatenated)
        #xafter= Dense(25,activation='sigmoid')(xcommun)


        #hyper0 = Dense(5, activation='sigmoid')(concatenated_hyper)
        hyper = Dense(1, activation='sigmoid', name='hyper_output')(xhyper)
        #hyper=Dropout(0.5)(hyper)


        model_hyper = Model(inputs=[inputs], outputs=[hyper])    


        #coord0 = Dense(5, activation='sigmoid')(concatenated_coord)
        coord = Dense(1, activation='sigmoid', name='coord_output')(xcoord)
        model_coord = Model(inputs=[inputs], outputs=[coord])





        model_coord.compile(optimizer='rmsprop', loss='binary_crossentropy')
        model_hyper.compile(optimizer='rmsprop', loss='binary_crossentropy')
        return model_coord, model_hyper

In [15]:
seed(1)
##############1
# Repeat the process with multi-task learning
num_epochs=3000
#seuil = 50
batch=64

#hyp=True
#Coord=True
import time


def train_model(data,name_model,Multi=False):
    tmps1=time.time()

    if Multi==False :
        model_coord = name_model()
        model_hyper = name_model()
    else : model_coord, model_hyper = name_model()




    acc_scores_coord,mif_scores_coord, acc_scores_hyper,  mif_scores_hyper= [], [], [], []
    precision_score_coord,precision_score_hyper,recall_score_hyper,recall_score_coord=[],[],[],[]

    for epoch in range(num_epochs):
        if epoch%500==0 :
            #print (epoch)
            tmps3=time.time()-tmps1
            #print ("Temps d'execution = %f" %tmps3 + "s")
        a1=len(data["Hyper-Random"]["x_train"])
        a2=len(data["Coord-Random"]["x_train"])

        idx1 = np.random.choice(np.arange(a1), batch, replace=False)
        idx2 = np.random.choice(np.arange(a2), batch, replace=False)
       
        model_hyper.fit(data["Hyper-Random"]["x_train"][idx1], np.array(data["Hyper-Random"]["y_train"])[idx1], epochs=1, validation_data=None, verbose=False, )
        model_coord.fit(data["Coord-Random"]["x_train"][idx2], np.array(data["Coord-Random"]["y_train"])[idx2], epochs=1, validation_data=None, verbose=False, )
        if epoch%200==0 :
            preds_coord = (model_coord.predict(data["Coord-Random"]["x_test"], verbose=0)> 0.5).astype(int)
            preds_hyper = (model_hyper.predict(data["Hyper-Random"]["x_test"], verbose=0)> 0.5).astype(int)

            preds_coord_valid = (model_coord.predict(data["Coord-Random"]["x_valid"], verbose=0)> 0.5).astype(int)
            preds_hyper_valid = (model_hyper.predict(data["Hyper-Random"]["x_valid"], verbose=0)> 0.5).astype(int)

            acc_scores_coord.append([accuracy_score(data["Coord-Random"]["y_valid"], preds_coord_valid), accuracy_score(data["Coord-Random"]["y_test"], preds_coord)])
            precision_score_coord.append([precision_score(data["Coord-Random"]["y_valid"], preds_coord_valid), precision_score(data["Coord-Random"]["y_test"], preds_coord)])
            recall_score_coord.append([recall_score(data["Coord-Random"]["y_valid"], preds_coord_valid), recall_score(data["Coord-Random"]["y_test"], preds_coord)])
            acc_scores_hyper.append([accuracy_score(data["Hyper-Random"]["y_valid"], preds_hyper_valid), accuracy_score(data["Hyper-Random"]["y_test"], preds_hyper)])
            precision_score_hyper.append([precision_score(data["Hyper-Random"]["y_valid"], preds_hyper_valid), precision_score(data["Hyper-Random"]["y_test"], preds_hyper)])
            recall_score_hyper.append([recall_score(data["Hyper-Random"]["y_valid"], preds_hyper_valid), recall_score(data["Hyper-Random"]["y_test"], preds_hyper)])

            mif_scores_coord.append([f1_score(data["Coord-Random"]["y_valid"], preds_coord_valid, average="macro") , f1_score(data["Coord-Random"]["y_test"], preds_coord, average="macro")])
            mif_scores_hyper.append([f1_score(data["Hyper-Random"]["y_valid"], preds_hyper_valid, average="macro") , f1_score(data["Hyper-Random"]["y_test"], preds_hyper, average="macro")])

    df1=pd.DataFrame(acc_scores_coord, columns = ['acc_coord_valid', 'acc_coord_test'])
    df2=pd.DataFrame(mif_scores_coord, columns = ['mif_coord_valid', 'mif_coord_test'])
    df3=pd.DataFrame(acc_scores_hyper, columns = ['acc_hyper_valid', 'acc_hyper_test'])
    df4=pd.DataFrame(mif_scores_hyper, columns = ['mif_hyper_valid', 'mif_hyper_test'])
    df5=pd.DataFrame(precision_score_coord, columns = ['prec_coord_valid', 'prec_coord_test'])
    df6=pd.DataFrame(precision_score_hyper, columns = ['prec_hyper_valid', 'prec_hyper_test'])
    df7=pd.DataFrame(recall_score_coord, columns = ['recall_coord_valid', 'recall_coord_test'])
    df8=pd.DataFrame(recall_score_hyper, columns = ['recall_hyper_valid', 'recall_hyper_test'])
    frames=[df1,df2,df3,df4,df5,df6,df7,df8]
    df_final=pd.concat(frames,axis=1)
    acc_coord=df_final['acc_coord_test'][np.argmax(df_final['acc_coord_valid'])]
    acc_hyper=df_final['acc_hyper_test'][np.argmax(df_final['acc_hyper_valid'])]
    mif_coord=df_final['mif_coord_test'][np.argmax(df_final['mif_coord_valid'])]
    mif_hyper=df_final['mif_hyper_test'][np.argmax(df_final['mif_hyper_valid'])]
    prec_coord=df_final['prec_coord_test'][np.argmax(df_final['prec_coord_valid'])]
    prec_hyper=df_final['prec_hyper_test'][np.argmax(df_final['prec_hyper_valid'])]
    recall_coord=df_final['recall_coord_test'][np.argmax(df_final['prec_coord_valid'])]
    recall_hyper=df_final['recall_hyper_test'][np.argmax(df_final['prec_hyper_valid'])]
    #recall_coord=df_final['recall_coord_test'][np.argmax(df_final['recall_coord_valid'])]
    #recall_hyper=df_final['recall_hyper_test'][np.argmax(df_final['recall_hyper_valid'])]
    tmps2=time.time()-tmps1
    tmps2=tmps2/60
    #new_data=[[acc_coord,'Nan'],[acc_hyper,'Nan'],[mif_coord,'Nan'],[mif_coord,'Nan'],[prec_coord,'Nan'],[prec_hyper,'Nan'],[recall_coord,'Nan'],[recall_hyper,'Nan'],[tmps2,'Nan']]
    new_data=[acc_coord,acc_hyper,mif_coord,mif_hyper,prec_coord,prec_hyper,recall_coord,recall_hyper,tmps2]
    df_restit=pd.DataFrame(new_data,index=['acc_coord','acc_hyp','mif_coord','mif_hyper','prec_coord','prec_hyper','recall_coord','recall_hyper','time'])
    tmps2=time.time()-tmps1
    tmps2=tmps2/60
    #print ("Temps d'execution = %f" %tmps2 + "min")

    return df_final ,df_restit.transpose()
# Monitor validation score in the score tuple and print accordingly
#print("NN baseline (Coord-Random):", max(acc_scores_coord2)[1], max(mif_scores_coord)[1], "\nNN Baseline (Hyper-Random):",  max(acc_scores_hyper2)[1], max(mif_scores_hyper)[1])
#print("Multitask (Coord-Random):", max(acc_scores_coord)[1], max(mif_scores_coord)[1])




In [16]:
def main(name_data,name_model,Multi,index):
    evol=[0,1,2,3,4,5]
    index_bool=['FTF','FTT','TFF','TFT','TTF','TTT']


    index_glob=[]
    for i in range(len(index)):
        for j in range(len(index_bool)):
            index_glob.append(index[i]+' '+index_bool[j])
    cols=['acc_coord','acc_hyp','maf_coord','maf_hyper','prec_coord','prec_hyper','recall_coord','recall_hyper','time']
    result=[]
    global concat
    global diff
    global cosine
    for i in range(len(name_model)):
        for j in range(len(evol)):
            if j==0:
                concat,diff,cosine=False,True,False
            if j==1:
                concat,diff,cosine=False,True,True
            if j==2:
                concat,diff,cosine=True,False,False
            if j==3:
                concat,diff,cosine=True,False,True
            if j==4:
                concat,diff,cosine=True,True,False
            if j==5:
                concat,diff,cosine=True,True,True
            data=func_train_test(name_data,embeddings,concat,diff,cosine)
    
            
            df_final,df_restit=train_model(data,name_model[i],Multi[i])
            result.append(list(df_restit.values)[0])
    
    df=pd.DataFrame(result,columns=cols,index=index_glob)
    return df
        
        


In [18]:

index=['NN baseline','Multi_task','Multi_task New','Multi Task New2']
models=[get_my_nn_model,get_my_multitask_nn_models_ancien,get_my_multitask_nn_models,get_my_multitask_nn_models_new]
list_Multi=[False,True,True,True]

In [419]:
df1=main("Rumen",models,list_Multi,index)
df1

/usr/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,acc_coord,acc_hyp,mif_coord,mif_hyper,prec_coord,prec_hyper,recall_coord,recall_hyper,time
NN baseline FTF,0.670301,0.699297,0.668152,0.695149,0.643238,0.645455,0.755486,0.674355,2.944852
NN baseline FTT,0.812046,0.770809,0.811863,0.767317,0.827283,0.727632,0.785789,0.750339,2.957698
NN baseline TFF,0.758048,0.770809,0.758027,0.767961,0.750000,0.721795,0.774295,0.763908,3.027496
NN baseline TFT,0.815161,0.794842,0.815158,0.792210,0.812695,0.749035,0.816092,0.789688,3.122538
NN baseline TTF,0.759605,0.781360,0.759565,0.778405,0.747454,0.735526,0.766980,0.758480,3.148857
NN baseline TTT,0.819834,0.791911,0.819834,0.789040,0.814433,0.666667,0.825496,0.027137,3.212144
Multi_task FTF,0.685358,0.710434,0.682888,0.710724,0.660309,0.653846,0.759666,0.715061,3.045831
Multi_task FTT,0.793873,0.757327,0.793861,0.753807,0.784553,0.711111,0.806688,0.738128,3.070011
Multi_task TFF,0.751817,0.790152,0.751790,0.787228,0.742164,0.746424,0.766980,0.778833,3.110412
Multi_task TFT,0.816719,0.794842,0.816713,0.792040,0.808163,0.753623,0.827586,0.776119,3.090714


In [ ]:
df2=main("Root9",models,list_Multi,index)
df2

/usr/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,acc_coord,acc_hyp,mif_coord,mif_hyper,prec_coord,prec_hyper,recall_coord,recall_hyper,time
NN baseline FTF,0.849260,0.779242,0.790294,0.730104,0.938247,0.839450,0.832155,0.851163,3.000263
NN baseline FTT,0.854643,0.787479,0.816429,0.720257,0.943798,0.838407,0.860424,0.832558,3.084697
NN baseline TFF,0.907133,0.833608,0.874703,0.803976,0.947748,0.898305,0.929329,0.862791,3.158659
NN baseline TFT,0.915209,0.843493,0.885173,0.812735,0.951439,0.895981,0.934629,0.881395,3.060039
NN baseline TTF,0.895020,0.833608,0.859648,0.778410,0.943636,0.881643,0.916961,0.848837,3.229066
NN baseline TTT,0.900404,0.830313,0.866247,0.798870,0.950459,0.892086,0.915194,0.865116,3.181862
Multi_task FTF,0.826380,0.803954,0.762169,0.757357,0.915851,0.851016,0.826855,0.876744,3.074163
Multi_task FTT,0.855989,0.792422,0.811164,0.753576,0.925788,0.863636,0.881625,0.839535,3.190025
Multi_task TFF,0.885599,0.836903,0.848945,0.806681,0.942910,0.896882,0.904594,0.869767,3.366650
Multi_task TFT,0.893674,0.833608,0.857591,0.799257,0.941924,0.888626,0.916961,0.872093,3.145937


In [19]:
df3=main("Weeds",models,list_Multi,index)
df3

/usr/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,acc_coord,acc_hyp,mif_coord,mif_hyper,prec_coord,prec_hyper,recall_coord,recall_hyper,time
NN baseline FTF,0.595876,0.694656,0.545589,0.694322,0.650146,0.722222,0.743333,0.687500,0.392020
NN baseline FTT,0.606186,0.697201,0.551024,0.696564,0.653521,0.716346,0.773333,0.716346,0.382573
NN baseline TFF,0.606186,0.669211,0.533522,0.668436,0.620309,0.712871,0.936667,0.692308,0.416552
NN baseline TFT,0.593814,0.681934,0.532062,0.680311,0.612691,0.694836,0.933333,0.711538,0.423071
NN baseline TTF,0.618557,0.709924,0.535333,0.708927,0.618557,0.700980,1.000000,0.687500,0.488061
NN baseline TTT,0.583505,0.704835,0.536718,0.683739,0.616667,0.719048,0.863333,0.725962,0.511418
Multi_task FTF,0.635052,0.712468,0.576753,0.711722,0.668493,0.731707,0.813333,0.721154,0.406805
Multi_task FTT,0.616495,0.681934,0.552394,0.681802,0.646154,0.715026,0.840000,0.663462,0.408623
Multi_task TFF,0.618557,0.709924,0.544720,0.709093,0.618557,0.728155,1.000000,0.721154,0.461954
Multi_task TFT,0.618557,0.699746,0.560586,0.698714,0.618557,0.716346,1.000000,0.716346,0.463696


In [20]:
df4=main("Bless",models,list_Multi,index)
df4

/usr/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,acc_coord,acc_hyp,mif_coord,mif_hyper,prec_coord,prec_hyper,recall_coord,recall_hyper,time
NN baseline FTF,0.832016,0.868626,0.810547,0.698599,0.767347,0.515152,0.727273,0.435897,0.660696
NN baseline FTT,0.861660,0.887640,0.841103,0.729198,0.837363,0.612069,0.736944,0.455128,0.689739
NN baseline TFF,0.843874,0.875540,0.821541,0.705644,0.806100,0.685950,0.715667,0.532051,1.141320
NN baseline TFT,0.856390,0.895419,0.835935,0.753424,0.824561,0.701031,0.727273,0.435897,0.698272
NN baseline TTF,0.846509,0.873812,0.824177,0.693915,0.808696,0.558559,0.719536,0.397436,0.777855
NN baseline TTT,0.864295,0.889369,0.841787,0.747690,0.837310,0.604478,0.746615,0.519231,0.785926
Multi_task FTF,0.824111,0.870354,0.798520,0.707769,0.771739,0.564885,0.686654,0.474359,0.689721
Multi_task FTT,0.870883,0.898876,0.851208,0.761576,0.855530,0.658537,0.733075,0.519231,0.696256
Multi_task TFF,0.837945,0.889369,0.810687,0.733364,0.818824,0.620690,0.673114,0.461538,0.761054
Multi_task TFT,0.859025,0.900605,0.836448,0.771285,0.842342,0.681416,0.723404,0.493590,0.751390


In [21]:
df5=main("Cogalex",models,list_Multi,index)
df5

/usr/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,acc_coord,acc_hyp,mif_coord,mif_hyper,prec_coord,prec_hyper,recall_coord,recall_hyper,time
NN baseline FTF,0.921569,0.868448,0.482870,0.604576,0.000000,0.336538,0.000000,0.275591,0.951495
NN baseline FTT,0.883442,0.875642,0.589004,0.700011,0.246377,0.530612,0.236111,0.409449,0.893490
NN baseline TFF,0.920479,0.866393,0.484693,0.540191,0.057692,0.000000,0.083333,0.000000,0.937315
NN baseline TFT,0.918301,0.869476,0.539278,0.602974,0.080000,0.500000,0.055556,0.023622,0.954138
NN baseline TTF,0.908497,0.855087,0.467570,0.553728,0.015385,0.252874,0.013889,0.173228,1.043518
NN baseline TTT,0.912854,0.852004,0.500793,0.589930,0.080460,0.380282,0.097222,0.212598,1.051141
Multi_task FTF,0.921569,0.869476,0.490733,0.584305,0.062500,0.448276,0.069444,0.102362,0.936677
Multi_task FTT,0.896514,0.875642,0.613589,0.693586,0.290909,0.615385,0.222222,0.251969,0.960076
Multi_task TFF,0.916122,0.865365,0.499121,0.506006,0.076923,0.140351,0.069444,0.062992,0.986986
Multi_task TFT,0.919390,0.839671,0.566981,0.569137,0.205882,0.306667,0.194444,0.181102,1.001340
